In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
# train.describe().T

In [ ]:
# train.info()

In [ ]:
#enc = preprocessing.OrdinalEncoder()
#train.target = enc.fit_transform(train[['target']])

In [ ]:
# test.describe().T

In [ ]:
# test.info()

In [ ]:
sample_submission.head()

In [ ]:
y_train = train['target']
X_train = train.drop(['id','target'], axis=1)
X_test = test.drop(['id'], axis=1)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.figure(figsize=(10,20))
sns.boxplot(data=X_train, orient="h");

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

scaler = preprocessing.StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

from sklearn.model_selection import train_test_split

X_train_trn, X_train_tst, y_train_trn, y_train_tst = train_test_split( X_train, y_train, test_size=0.1)

In [ ]:

# Precision: 
    # tp / (tp + fp)
    # fp = typeI error
    # True positive / Predicted condition positive
    # Among the ones predicted as certain label, actually correct ones.
    # The precision will be "how many are correctly classified among that class"
    # “for all instances classified positive, what percent was correct?”
    # Precision is the ability of a classifier not to label an instance positive that is actually negative. 

# Recall/Sensitivity/Power of a Test:
    # tp / (tp + fn)
    # fn = actual label does not belongs predicted label.
    # True Predicted Positive / All Real Positives 
    # Correctly identifying (true pos) a label when sample belongs to that (all pos). 
    # Proportion of positives that are correctly identified
    # TPR = true positive rate
    # The recall means "how many of this class you find over the whole number of element of this class
    # What percent of the positive cases did you catch?
    
# Specificity =(True Negative rate) 

# F1 = 2 * (precision * recall) / (precision + recall)
# F1 score reaches its best value at 1 and worst score at 0. 
# What percent of positive predictions were correct?

# support in classification report in reality true instances.  The support is the number of occurrences of each class in y_true.

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

clf_HistGradientBoostingClassifier = HistGradientBoostingClassifier()

clf_HistGradientBoostingClassifier.fit(X_train, y_train)
#predictions_b = clf_HistGradientBoostingClassifier.predict(X_test)
predictions_p = clf_HistGradientBoostingClassifier.predict_proba(X_test)
sample_submission.iloc[:,1:5] = predictions_p
sample_submission.to_csv('submission_HistGradientBoostingClassifier.csv',index=False)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf_HistGradientBoostingClassifier, X_train, y_train, cv=5, scoring='neg_log_loss')
print('Log_loss scores:', -scores)
#from sklearn.metrics import classification_report, log_loss

#print('Simple clf.score method of the estimator:', clf_HistGradientBoostingClassifier.score(X_train, y_train), '\n\n')
#print('Detailed Classification Report: \n\n',classification_report(y_train, clf_HistGradientBoostingClassifier.predict(X_train), zero_division=0))
#print('LogLoss:',log_loss(y_train, clf_HistGradientBoostingClassifier.predict_proba(X_train)))

In [ ]:
# Understanding Multi-class logarithmic loss with Pandas

df1 = pd.DataFrame(clf_HistGradientBoostingClassifier.predict_proba(X_train), columns=['Class_1','Class_2','Class_3','Class_4'])
df1['actuals'] = y_train
df1['predictions'] = clf_HistGradientBoostingClassifier.predict(X_train)
df1['nl1']=-np.log(df1['Class_1'])
df1['nl2']=-np.log(df1['Class_2'])
df1['nl3']=-np.log(df1['Class_3'])
df1['nl4']=-np.log(df1['Class_4'])
df1['Prediction Correct ?'] = (df1['actuals'] == df1['predictions'])*1
df1['nl_final'] = df1['nl1']*(df1['actuals']=='Class_1')+df1['nl2']*(df1['actuals']=='Class_2')+df1['nl3']*(df1['actuals']=='Class_3')+df1['nl4']*(df1['actuals']=='Class_4')
print('Correct Prediction #:',df1['Prediction Correct ?'].sum())
print('Correct Prediction %:',df1['Prediction Correct ?'].sum()/len(df1))
print('nl_final_average:',df1['nl_final'].sum()/len(df1))

df1

from lightgbm import LGBMClassifier
clf_LGBMClassifier = LGBMClassifier()
clf_LGBMClassifier.fit(X_train, y_train)
predictions_p = clf_LGBMClassifier.predict_proba(X_test)
sample_submission.iloc[:,1:5] = predictions_p
sample_submission.to_csv('submission_LGBMClassifier.csv',index=False)

scores = cross_val_score(clf_LGBMClassifier, X_train, y_train, cv=5, scoring='neg_log_loss')
print('Log_loss scores:', -scores)

In [ ]:
from catboost import CatBoostClassifier
clf_CatBoostClassifier = CatBoostClassifier(verbose=0)
clf_CatBoostClassifier.fit(X_train, y_train)
predictions_p = clf_CatBoostClassifier.predict_proba(X_test)
sample_submission.iloc[:,1:5] = predictions_p
sample_submission.to_csv('submission_CatBoostClassifier.csv',index=False)

scores = cross_val_score(clf_CatBoostClassifier, X_train, y_train, cv=5, scoring='neg_log_loss')
print('Log_loss scores:', -scores)


from xgboost import XGBClassifier
clf_XGBClassifier = XGBClassifier() 
clf_XGBClassifier.fit(X_train, y_train)
predictions_p = clf_XGBClassifier.predict_proba(X_test)
sample_submission.iloc[:,1:5] = predictions_p
sample_submission.to_csv('submission_XGBClassifier.csv',index=False)

scores = cross_val_score(clf_XGBClassifier, X_train, y_train, cv=5, scoring='neg_log_loss')
print('Log_loss scores:', -scores)

from sklearn.neural_network import MLPClassifier
clf_MLPClassifier = MLPClassifier(random_state=1, max_iter=1000)
clf_MLPClassifier.fit(X_train, y_train)
predictions_p = clf_MLPClassifier.predict_proba(X_test)
sample_submission.iloc[:,1:5] = predictions_p
sample_submission.to_csv('submission_MLPClassifier.csv',index=False)

scores = cross_val_score(clf_MLPClassifier, X_train, y_train, cv=5, scoring='neg_log_loss')
print('Log_loss scores:', -scores)